<a href="https://colab.research.google.com/github/c-c-c-c/kaggle_titanic/blob/master/myWrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#追記
import json
import datetime
import math

#importじゃないけど、個人的には省略されて良いことがあまりないので・・
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [0]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kaggle_titanic/titanic/data/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kaggle_titanic/titanic/data/test.csv')
gender_submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kaggle_titanic/titanic/data/gender_submission.csv')
# gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

In [14]:
train.head(10)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1
5,0,3,NaN,0,0,8.4583,0,1,0,1,0
6,0,1,54.0,0,0,51.8625,0,1,0,0,1
7,0,3,2.0,3,1,21.0750,0,1,0,0,1
8,1,3,27.0,0,2,11.1333,1,0,0,0,1
9,1,2,14.0,1,0,30.0708,1,0,1,0,0


In [0]:
# SexとEmbarkedのOne-Hotエンコーディング
train = pd.get_dummies(train, columns=['Sex', 'Embarked']) 
test = pd.get_dummies(test, columns=['Sex', 'Embarked'])
# 不要な列の削除
train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

In [13]:
feat_cols =  list(train.columns).remove('Survived')

categorical_feature = ["Pclass", 

['Survived',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Sex_female',
 'Sex_male',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

In [0]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            # 'metric': {'rmse'},
            'subsample': 0.7,
            'subsample_freq': 1,
            'learning_rate': 0.01,
            'num_leaves': 100,
            'min_data_in_leaf':24,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1
            }

In [0]:
folds = 5
seed = 666

kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

In [0]:
models = []
for train_index, val_index in kf.split(train, train['survived']):
    train_X = train[feat_cols].iloc[train_index]
    val_X = train[feat_cols].iloc[val_index]
    train_y = train["Survived"].iloc[train_index]
    val_y = train["Survived"].iloc[val_index]

    lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
    lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)

    
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)
    models.append(gbm)